https://github.com/ThomasDelteil/DeepLearningWithMXNetGluon/blob/master/9_object_segmentation.ipynb

In [1]:
!pip install -U mxnet-cu101==1.7.0
!pip install d2l==0.16.0

     |███████████████████████████████▌| 834.1MB 1.2MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x65116000 @  0x7efccb5c4615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 19kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 81kB 4.4MB/s 


In [2]:
!pip install gluoncv

     |████████████████████████████████| 1.0MB 6.4MB/s 
     |████████████████████████████████| 11.8MB 13.6MB/s 
     |████████████████████████████████| 317kB 47.7MB/s 
     |████████████████████████████████| 256kB 34.4MB/s 
     |████████████████████████████████| 102kB 6.4MB/s 
     |████████████████████████████████| 102kB 6.2MB/s 
     |████████████████████████████████| 215kB 46.3MB/s 
     |████████████████████████████████| 675kB 42.2MB/s 
     |████████████████████████████████| 972kB 41.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.2MB 39.5MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 962kB 37.1MB/s 
     |████████████████████████████████| 2.6MB 40.1MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
  Created wheel for ConfigSpace: filenam

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

import random, multiprocessing, time

import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
from mxnet import gluon, autograd
from mxnet.gluon import nn

from mxboard import SummaryWriter

import numpy as np

import gluoncv
from gluoncv.utils.parallel import *
from gluoncv.utils.viz import get_color_pallete, DeNormalize

ModuleNotFoundError: ignored

In [4]:
!pip install mxboard

     |████████████████████████████████| 51kB 3.0MB/s 


In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

import random, multiprocessing, time

import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
from mxnet import gluon, autograd
from mxnet.gluon import nn

from mxboard import SummaryWriter

import numpy as np

import gluoncv
from gluoncv.utils.parallel import *
from gluoncv.utils.viz import get_color_pallete, DeNormalize

In [6]:
nclasses = len(gluoncv.data.VOCSegmentation().classes)

OSError: ignored

In [7]:
nclasses = len(gluoncv.data.VOCSegmentation().classes)

OSError: ignored

In [8]:
nclasses = len(gluoncv.data.VOCDetection().classes)

OSError: ignored

In [9]:
gluoncv.data.VOCDetection()

OSError: ignored

In [10]:
gluoncv.data.Kinetics400()

RuntimeError: ignored

In [11]:
train_dataset = gluoncv.data.VOCDetection(splits=[(2007, 'trainval'), (2012, 'trainval')])

OSError: ignored

In [12]:
import os

In [13]:
os.makedirs(~/.mxnet/datasets/voc)

SyntaxError: ignored

In [14]:
os.makedirs(~\.mxnet\datasets\voc)

SyntaxError: ignored

In [15]:
os.makedirs(~\.mxnet\datasets\voc)

SyntaxError: ignored

In [16]:
os.makedirs(/.mxnet/datasets/voc)

SyntaxError: ignored

In [17]:
os.makedirs("~/.mxnet/datasets/voc")

In [18]:
nclasses = len(gluoncv.data.VOCSegmentation().classes)

OSError: ignored

In [19]:
os.makedirs("root/.mxnet/datasets/voc")

In [20]:
nclasses = len(gluoncv.data.VOCSegmentation().classes)

OSError: ignored

In [21]:
os.makedirs("usr/.mxnet/datasets/voc")

In [22]:
nclasses = len(gluoncv.data.VOCSegmentation(root='/usr/.mxnet/datasets/voc').classes)

OSError: ignored

In [23]:
_TARGET_DIR = os.path.expanduser('~/.mxnet/datasets/voc')

In [24]:
nclasses = len(gluoncv.data.VOCSegmentation().classes)

OSError: ignored

In [25]:
os.makedirs("usr/.mxnet/datasets/voc")

FileExistsError: ignored

In [26]:
nclasses = len(gluoncv.data.VOCSegmentation(root='usr/.mxnet/datasets/voc').classes)

FileNotFoundError: ignored

https://blog.csdn.net/weixin_41278720/article/details/83472401

In [27]:
_TARGET_DIR = os.path.expanduser('usr/.mxnet/datasets/voc')

In [28]:
# Download and extract VOC datasets into ``path``
 
def download_voc(path, overwrite=False):
    _DOWNLOAD_URLS = [
        ('http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar',
         '34ed68851bce2a36e2a223fa52c661d592c66b3c'),
        ('http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar',
         '41a8d6e12baa5ab18ee7f8f8029b9e11805b4ef1'),
        ('http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar',
         '4e443f8a2eca6b1dac8a6c57641b67dd40621a49')]
    makedirs(path)
    for url, checksum in _DOWNLOAD_URLS:
        filename = download(url, path=path, overwrite=overwrite, sha1_hash=checksum)
        # extract
        with tarfile.open(filename) as tar:
            tar.extractall(path=path)

In [29]:

#####################################################################################
# Download and extract the VOC augmented segmentation dataset into ``path``
 
def download_aug(path, overwrite=False):
    _AUG_DOWNLOAD_URLS = [
        ('http://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz', '7129e0a480c2d6afb02b517bb18ac54283bfaa35')]
    makedirs(path)
    for url, checksum in _AUG_DOWNLOAD_URLS:
        filename = download(url, path=path, overwrite=overwrite, sha1_hash=checksum)
        # extract
        with tarfile.open(filename) as tar:
            tar.extractall(path=path)
            shutil.move(os.path.join(path, 'benchmark_RELEASE'),
                        os.path.join(path, 'VOCaug'))
            filenames = ['VOCaug/dataset/train.txt', 'VOCaug/dataset/val.txt']
            # generate trainval.txt
            with open(os.path.join(path, 'VOCaug/dataset/trainval.txt'), 'w') as outfile:
                for fname in filenames:
                    fname = os.path.join(path, fname)
                    with open(fname) as infile:
                        for line in infile:
                            outfile.write(line)

In [30]:
download_aug('usr/.mxnet/datasets/voc', overwrite=args.overwrite)

NameError: ignored

In [31]:
download_aug('usr/.mxnet/datasets/voc')

NameError: ignored

In [32]:
"""Prepare PASCAL VOC datasets"""
import os
import shutil
import argparse
import tarfile
from gluoncv.utils import download, makedirs

In [33]:
download_aug('usr/.mxnet/datasets/voc')

1386270KB [05:03, 4575.04KB/s]                             


In [34]:
nclasses = len(gluoncv.data.VOCSegmentation().classes)

OSError: ignored

In [35]:
nclasses = len(gluoncv.data.VOCSegmentation('usr/.mxnet/datasets/voc').classes)

FileNotFoundError: ignored

In [36]:
path = os.path.expanduser('usr/.mxnet/datasets/voc')

In [37]:
    if not os.path.isdir(path) or not os.path.isdir(os.path.join(path, 'VOC2007')) \
        or not os.path.isdir(os.path.join(path, 'VOC2012')):
        if args.no_download:
            raise ValueError(('{} is not a valid directory, make sure it is present.'
                              ' Or you should not disable "--no-download" to grab it'.format(path)))
        else:
            download_voc(path, overwrite=args.overwrite)
            shutil.move(os.path.join(path, 'VOCdevkit', 'VOC2007'), os.path.join(path, 'VOC2007'))
            shutil.move(os.path.join(path, 'VOCdevkit', 'VOC2012'), os.path.join(path, 'VOC2012'))
            shutil.rmtree(os.path.join(path, 'VOCdevkit'))

NameError: ignored

In [38]:
if not os.path.isdir(path) or not os.path.isdir(os.path.join(path, 'VOC2007')) \
    or not os.path.isdir(os.path.join(path, 'VOC2012')):
    download_voc(path, overwrite=args.overwrite)
    shutil.move(os.path.join(path, 'VOCdevkit', 'VOC2007'), os.path.join(path, 'VOC2007'))
    shutil.move(os.path.join(path, 'VOCdevkit', 'VOC2012'), os.path.join(path, 'VOC2012'))
    shutil.rmtree(os.path.join(path, 'VOCdevkit'))

NameError: ignored

In [39]:
if not os.path.isdir(path) or not os.path.isdir(os.path.join(path, 'VOC2007')) \
    or not os.path.isdir(os.path.join(path, 'VOC2012')):
    download_voc(path)
    shutil.move(os.path.join(path, 'VOCdevkit', 'VOC2007'), os.path.join(path, 'VOC2007'))
    shutil.move(os.path.join(path, 'VOCdevkit', 'VOC2012'), os.path.join(path, 'VOC2012'))
    shutil.rmtree(os.path.join(path, 'VOCdevkit'))

100%|██████████| 449250/449250 [00:36<00:00, 12429.21KB/s]


100%|██████████| 440450/440450 [00:35<00:00, 12328.70KB/s]


100%|██████████| 1952773/1952773 [02:36<00:00, 12440.97KB/s]


In [40]:
nclasses = len(gluoncv.data.VOCSegmentation('usr/.mxnet/datasets/voc').classes)

In [41]:
class FCN(nn.HybridBlock):
    def __init__(self, resnet_backbone, nclass):
        super(FCN, self).__init__()
        self.backbone = resnet_backbone 
        in_channels = 2048
        with self.name_scope():
            self.head = nn.HybridSequential()
            inter_channels = in_channels // 4
            self.head.add(nn.Conv2D(in_channels=in_channels, channels=inter_channels,
                                     kernel_size=3, padding=1))
            self.head.add(nn.BatchNorm(in_channels=inter_channels))
            self.head.add(nn.Activation('relu'))
            self.head.add(nn.Dropout(0.1))
            self.head.add(nn.Conv2D(in_channels=inter_channels, channels=nclass,
                                     kernel_size=1))
        
    def hybrid_forward(self, F, x):
        # run through backbone
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        c3 = self.backbone.layer3(x)
        c4 = self.backbone.layer4(c3)

        #run through head
        y = self.head(c4)

        #resize
        y = F.contrib.BilinearResize2D(y, height=480,width=480)
        return y

In [42]:
ctx_list = [mx.gpu(i) for i in range(4)]
backbone = gluoncv.model_zoo.resnet50_v1b(pretrained=True, dilated=True, ctx=ctx_list)
del backbone.fc

net = FCN(backbone, nclasses)

# Initialize the head
net.head.load_parameters().setattr('lr_mult', 10)
net.head.initialize(mx.init.Xavier(), ctx=ctx_list)

100%|██████████| 55344/55344 [00:00<00:00, 60088.92KB/s]


MXNetError: ignored